In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# general lib
from tqdm import tqdm
import cv2

# models
from sklearn.neighbors import KNeighborsClassifier

# score metrics
from sklearn.metrics import f1_score

In [ ]:
# Loading the dataset , testset
train_df = pd.read_csv('/kaggle/input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train.csv')
test_df  = pd.read_csv('/kaggle/input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test.csv')

In [ ]:
# Appending .jpg against file name

train = train_df.copy()
test = test_df.copy()

train['image_id'] = train['image_id']+str('.jpg')
test['image_id'] = test['image_id']+str('.jpg')

In [ ]:
# converting and loading images in numpy format with predefined size
IMG_SIZE = 224

train_img=[]
train_label=[]
j=0
path='../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train'
for i in tqdm(train['image_id']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img=img.astype('float32')
    train_img.append(img.flatten())
    train_label.append(train['breed'][j])
    j=j+1

In [ ]:
len(train_img[0])

In [ ]:
# impoting test image same way as train
test_img=[]
path='../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test'
for i in tqdm(test['image_id']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img=img.astype('float32')
    test_img.append(img.flatten())

In [ ]:
# Splitting data for validation and accuracy testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_img, train_label, test_size=0.2, random_state=42)

In [ ]:
# garbage collector to free up ram
import gc
gc.collect()

In [ ]:
# KNN model
nclass = 35 # number of classes

neigh = KNeighborsClassifier(n_neighbors=nclass)
neigh.fit(X_train, y_train)
100*f1_score(y_test, pred, average='weighted')

In [ ]:
import gc
gc.collect()

In [ ]:
# for storing the model

import pickle 
knnPickle = open('knnpickle_file', 'wb') 

# store the model in bin format
pickle.dump(neigh, knnPickle)

# Load The model
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)

In [ ]:
# predicting on test data
pred = neigh.predict(test)
pred

In [ ]:
# converting predictions to dataframe
output = pd.DataFrame({'image_id': test.image_id, 'breed': pred})
filename = 'solution.csv'
output.to_csv(filename, index=False) 
output.head()